#### IMPORT LIBRARIES

In [1]:
import numpy as np
import h5py as h5

import torch
from torchvision import datasets, transforms
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.tensorboard import SummaryWriter

import matplotlib.pyplot as plt
from importlib import reload, import_module

import glob
import os

import pdb
from PIL import Image as im
import _pickle as pickle


from functions import MyDataset, customTransform, get_variable, get_numpy, compute_gradient, psnr_1, count_parameters

#### DATASET PATH

In [2]:
if os.name == 'nt':
    dataset_file = r"C:\Users\mummu\Documents\Datasets\srinivasan\trainset\h5\8bit.h5"
    test_file    = r"C:\Users\mummu\Documents\Datasets\srinivasan\testset\h5\8bit.h5"
    model_file   = r"model\model.pt"
    network_file = r"network_train"
    trainwr_file = r"runs\train"
    testwr_file  = r"runs\test"
elif os.name == 'posix':
    raise NotImplementedError

#### BASIC PARAMETERS

In [3]:
patch_size     = 192
batch_size     = 300
minibatch_size = 3
gamma_val      = 0.4
lfsize         = [372, 540, 7, 7]
num_workers    = 0
num_test       = 10
num_minibatch  = batch_size//minibatch_size
batch_affine   = True
num_epochs     = 10000

#### INITIALIZE FUNCTIONS

In [4]:
data_transform = transforms.Compose([transforms.ToTensor(), 
                                     transforms.Lambda(customTransform)])

train_dataset  = MyDataset(dataset_file, lfsize, data_transform)
test_dataset   = MyDataset(test_file, lfsize, data_transform)

train_loader   = torch.utils.data.DataLoader(train_dataset, batch_size=minibatch_size, num_workers=num_workers, shuffle=True)
test_loader    = torch.utils.data.DataLoader(train_dataset, batch_size=minibatch_size, num_workers=num_workers, shuffle=True)

#### LOOKING FOR SAVED MODEL

In [5]:
network_module = import_module(network_file)
reload(network_module)
Net = network_module.Net

net = Net((patch_size, patch_size), minibatch_size, lfsize, batchAffine=batch_affine)
if torch.cuda.is_available():
    print('##converting network to cuda-enabled')
    net.cuda()

try:
    checkpoint = torch.load(model_file)
    
    epoch_id = checkpoint['epoch']
    net.load_state_dict(checkpoint['model'].state_dict())
    print('Model successfully loaded.')
    
except:
    print('No model.')
    epoch_id = 0

##converting network to cuda-enabled
Model successfully loaded.


#### TRAINING SETTINGS

In [6]:
criterion1 = nn.L1Loss()
criterion2 = nn.L1Loss()
criterion3 = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=0.001, betas=(0.9, 0.999))

In [7]:
def train_epoch():
    costs = []
    psnr_vec = []
            
    for batch_num in range(num_minibatch):
        
        # fetching training batch
        corners, pers, ind = next(iter(train_loader))
        
        # converting to trainable variables
        X_corners = get_variable(corners)
        T_view = get_variable(pers)
        p = get_variable(ind[:,0])
        q = get_variable(ind[:,-1])
                
        optimizer.zero_grad()
        
        # Forward pass
        O_view, M = net(X_corners, p, q)
        
        # Computing batch loss
        batch_loss = criterion1(O_view, T_view) + .5*criterion2(compute_gradient(O_view), compute_gradient(T_view)) \
                    + 0.5*((M.reshape(-1,12).mean(0))**2).sum()
        
        # Backpropagation
        batch_loss.backward()
        optimizer.step()

        # recording performance
        costs.append(get_numpy(batch_loss))
        net_out = get_numpy(O_view)
        Y = get_numpy(T_view)      
        psnr_vec.append([psnr_1(np.squeeze(net_out[i]), np.squeeze(Y[i])) for i in range(minibatch_size)])
    
        
    return np.mean(costs), np.mean(psnr_vec)

def eval_epoch():
    costs = []
    psnr_vec = []
    
    for batch_num in range(num_test):
        
        # fetching training batch
        corners, pers, ind = next(iter(test_loader))
        
        # converting to trainable variables
        X_corners = get_variable(corners)
        T_view = get_variable(pers)
        p = get_variable(ind[:,0])
        q = get_variable(ind[:,-1])

        with torch.no_grad():
            # Forward pass
            O_view, M = net(X_corners, p, q)
            
            # Computing batch loss
            batch_loss = criterion1(O_view, T_view) + .5*criterion2(compute_gradient(O_view), compute_gradient(T_view))\
                    + 0.5*((M.reshape(-1,12).mean(0))**2).sum()
            
            # recording performance
            costs.append(get_numpy(batch_loss))
            net_out = get_numpy(O_view)
            Y = get_numpy(T_view)
            psnr_vec.append([psnr_1(np.squeeze(net_out[i]), np.squeeze(Y[i])) for i in range(minibatch_size)])

    return np.mean(costs), np.mean(psnr_vec)

In [ ]:
valid_accs, train_accs, test_accs = [], [], []

writer_train = SummaryWriter(trainwr_file)
writer_test  = SummaryWriter(testwr_file)

while epoch_id < num_epochs:
    epoch_id += 1
    
    try:   
        net.train()
        train_cost, train_psnr = train_epoch()
        
        net.eval()
        test_cost, test_psnr = eval_epoch()
        
        print("Epoch %d:" % epoch_id)     
        print("Epoch {0:0}, train_cost {1:.2}, psnr {2:.2}".format(epoch_id, train_cost, train_psnr))
        
        writer_train.add_scalar('psnr', train_psnr, epoch_id)
        writer_train.add_scalar('loss', train_cost, epoch_id)
        writer_test.add_scalar('psnr', test_psnr, epoch_id)
        writer_test.add_scalar('loss', test_cost, epoch_id)
        
        torch.save({'model': net, 'epoch': epoch_id}, model_file)
    
    except KeyboardInterrupt:
        print('\nKeyboardInterrupt')
        break

C:\Users\mummu\AppData\Local\Continuum\anaconda3\lib\site-packages\torch\nn\functional.py:2404: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


Epoch 1192:
Epoch 1192, train_cost 0.082, psnr 4e+01


C:\Users\mummu\AppData\Local\Continuum\anaconda3\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\mummu\AppData\Local\Continuum\anaconda3\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Conv2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\mummu\AppData\Local\Continuum\anaconda3\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type AvgPool2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\mummu\AppData\Local\Continuum\anaconda3\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type GroupNorm. It won't be checked

Epoch 1193:
Epoch 1193, train_cost 0.08, psnr 4.1e+01
Epoch 1194:
Epoch 1194, train_cost 0.081, psnr 4e+01
Epoch 1195:
Epoch 1195, train_cost 0.082, psnr 4e+01
Epoch 1196:
Epoch 1196, train_cost 0.083, psnr 4e+01
Epoch 1197:
Epoch 1197, train_cost 0.081, psnr 4.1e+01
Epoch 1198:
Epoch 1198, train_cost 0.082, psnr 4e+01
Epoch 1199:
Epoch 1199, train_cost 0.082, psnr 4e+01
Epoch 1200:
Epoch 1200, train_cost 0.079, psnr 4.1e+01
Epoch 1201:
Epoch 1201, train_cost 0.081, psnr 4e+01
Epoch 1202:
Epoch 1202, train_cost 0.081, psnr 4e+01
Epoch 1203:
Epoch 1203, train_cost 0.082, psnr 4e+01
Epoch 1204:
Epoch 1204, train_cost 0.081, psnr 4e+01
Epoch 1205:
Epoch 1205, train_cost 0.081, psnr 4e+01
Epoch 1206:
Epoch 1206, train_cost 0.081, psnr 4.1e+01
Epoch 1207:
Epoch 1207, train_cost 0.083, psnr 4e+01
Epoch 1208:
Epoch 1208, train_cost 0.08, psnr 4e+01
Epoch 1209:
Epoch 1209, train_cost 0.082, psnr 4e+01
Epoch 1210:
Epoch 1210, train_cost 0.079, psnr 4.1e+01
Epoch 1211:
Epoch 1211, train_cost 0.0

Epoch 1345, train_cost 0.079, psnr 4.1e+01
Epoch 1346:
Epoch 1346, train_cost 0.083, psnr 4e+01
Epoch 1347:
Epoch 1347, train_cost 0.081, psnr 4e+01
Epoch 1348:
Epoch 1348, train_cost 0.081, psnr 4.1e+01
Epoch 1349:
Epoch 1349, train_cost 0.081, psnr 4e+01
Epoch 1350:
Epoch 1350, train_cost 0.08, psnr 4.1e+01
Epoch 1351:
Epoch 1351, train_cost 0.082, psnr 4e+01
Epoch 1352:
Epoch 1352, train_cost 0.081, psnr 4.1e+01
Epoch 1353:
Epoch 1353, train_cost 0.082, psnr 4e+01
Epoch 1354:
Epoch 1354, train_cost 0.082, psnr 4e+01
Epoch 1355:
Epoch 1355, train_cost 0.082, psnr 4e+01
Epoch 1356:
Epoch 1356, train_cost 0.08, psnr 4.1e+01
Epoch 1357:
Epoch 1357, train_cost 0.082, psnr 4e+01
Epoch 1358:
Epoch 1358, train_cost 0.082, psnr 4e+01
Epoch 1359:
Epoch 1359, train_cost 0.083, psnr 4e+01
Epoch 1360:
Epoch 1360, train_cost 0.083, psnr 4e+01
Epoch 1361:
Epoch 1361, train_cost 0.082, psnr 4e+01
Epoch 1362:
Epoch 1362, train_cost 0.083, psnr 4e+01
Epoch 1363:
Epoch 1363, train_cost 0.081, psnr 4e+

Epoch 1497, train_cost 0.082, psnr 4e+01
Epoch 1498:
Epoch 1498, train_cost 0.082, psnr 4e+01
Epoch 1499:
Epoch 1499, train_cost 0.081, psnr 4.1e+01
Epoch 1500:
Epoch 1500, train_cost 0.081, psnr 4.1e+01
Epoch 1501:
Epoch 1501, train_cost 0.082, psnr 4e+01
Epoch 1502:
Epoch 1502, train_cost 0.082, psnr 4e+01
Epoch 1503:
Epoch 1503, train_cost 0.081, psnr 4e+01
Epoch 1504:
Epoch 1504, train_cost 0.081, psnr 4e+01
Epoch 1505:
Epoch 1505, train_cost 0.081, psnr 4.1e+01
Epoch 1506:
Epoch 1506, train_cost 0.081, psnr 4e+01
Epoch 1507:
Epoch 1507, train_cost 0.081, psnr 4e+01
Epoch 1508:
Epoch 1508, train_cost 0.083, psnr 4e+01
Epoch 1509:
Epoch 1509, train_cost 0.08, psnr 4.1e+01
Epoch 1510:
Epoch 1510, train_cost 0.082, psnr 4e+01
Epoch 1511:
Epoch 1511, train_cost 0.082, psnr 4e+01
Epoch 1512:
Epoch 1512, train_cost 0.081, psnr 4.1e+01
Epoch 1513:
Epoch 1513, train_cost 0.082, psnr 4e+01
Epoch 1514:
Epoch 1514, train_cost 0.081, psnr 4.1e+01
Epoch 1515:
Epoch 1515, train_cost 0.08, psnr 4

In [ ]:
from network import img_diff, img_show, img_disp